In [ ]:
import pprint
import pandas as pd
import numpy as np
import evo.core.trajectory as et
from evo.core import metrics
from evo.core.units import Unit
import evo.tools.plot as ep
from evo.tools.pandas_bridge import df_to_trajectory
import matplotlib.pyplot as plt
from scipy.spatial.transform import Rotation as R

In [ ]:
from evo.tools import log
log.configure_logging(verbose=True, debug=True, silent=False)
from evo.core import sync
# temporarily override some package settings
from evo.tools.settings import SETTINGS
SETTINGS.plot_usetex = False

ep.apply_settings(SETTINGS)
%matplotlib inline

In [ ]:
import os
import sys

In [ ]:
# Load the files
scene = "individual_files_validation_segment-10289507859301986274_4200_000_4220_000_with_camera_labels"
date = "202503151829"

# filepath of the csv file with the scene poses_data
poses_file_path = os.path.join(os.getcwd(), scene + "/" + date, "poses_" + scene + ".csv")
landmarks_file_path = os.path.join(os.getcwd(), scene + "/" + date, "landmarks_" + scene + ".csv")
print("Poses File Path:", poses_file_path)
print("Landmarks File Path:", landmarks_file_path)

In [ ]:
data = pd.read_csv(poses_file_path)
data.shape

In [ ]:
data.head()

In [ ]:
# Generar timestamps (10Hz)
timestamps = np.arange(len(data)) * 0.1

In [ ]:
gt_positions = data[['real_x', 'real_y', 'real_z']].values
odometry_positions = data[['odometry_x', 'odometry_y', 'odometry_z']].values
estimated_positions = data[['corrected_x', 'corrected_y', 'corrected_z']].values

In [ ]:
gt_orientations = data[['real_roll', 'real_pitch', 'real_yaw']].values
odometry_orientations = data[['odometry_roll', 'odometry_pitch', 'odometry_yaw']].values
estimated_orientations = data[['corrected_roll', 'corrected_pitch', 'corrected_yaw']].values

In [ ]:
# Convertir ángulos de Euler a cuaterniones (formato w, x, y, z)
gt_rotations = R.from_euler('xyz', gt_orientations, degrees=False)
gt_quaternions = gt_rotations.as_quat()  # Devuelve (x, y, z, w), hay que reordenar a (w, x, y, z)
gt_quaternions = np.roll(gt_quaternions, shift=1, axis=1)  # Mover el último elemento al inicio

odometry_orientations = R.from_euler('xyz', odometry_orientations, degrees=False)
odometry_quaternions = odometry_orientations.as_quat()
odometry_quaternions = np.roll(odometry_quaternions, shift=1, axis=1)

estimated_orientations = R.from_euler('xyz', estimated_orientations, degrees=False)
estimated_quaternions = estimated_orientations.as_quat()
estimated_quaternions = np.roll(estimated_quaternions, shift=1, axis=1)


In [ ]:
# Crear un DataFrame en formato evo
df_gt_evo = pd.DataFrame({
    "x": gt_positions[:, 0],
    "y": gt_positions[:, 1],
    "z": gt_positions[:, 2],
    "qw": gt_quaternions[:, 0],
    "qx": gt_quaternions[:, 1],
    "qy": gt_quaternions[:, 2],
    "qz": gt_quaternions[:, 3],
}, index=timestamps)

df_odometry_evo = pd.DataFrame({
    "x": odometry_positions[:, 0],
    "y": odometry_positions[:, 1],
    "z": odometry_positions[:, 2],
    "qw": odometry_quaternions[:, 0],
    "qx": odometry_quaternions[:, 1],
    "qy": odometry_quaternions[:, 2],
    "qz": odometry_quaternions[:, 3],
}, index=timestamps)

df_estimated_evo = pd.DataFrame({
    "x": estimated_positions[:, 0],
    "y": estimated_positions[:, 1],
    "z": estimated_positions[:, 2],
    "qw": estimated_quaternions[:, 0],
    "qx": estimated_quaternions[:, 1],
    "qy": estimated_quaternions[:, 2],
    "qz": estimated_quaternions[:, 3],
}, index=timestamps)

Trajectories

In [ ]:
# Convertir DataFrame a PoseTrajectory3D
traj_gt = df_to_trajectory(df_gt_evo, as_type=et.PoseTrajectory3D)
traj_odometry = df_to_trajectory(df_odometry_evo, as_type=et.PoseTrajectory3D)
traj_estimated = df_to_trajectory(df_estimated_evo, as_type=et.PoseTrajectory3D)

In [ ]:
max_diff = 0.01
traj_gt, traj_estimated = sync.associate_trajectories(traj_gt, traj_estimated, max_diff)

In [ ]:

import copy

traj_est_aligned = copy.deepcopy(traj_estimated)
traj_est_aligned.align(traj_gt, correct_scale=False, correct_only_scale=False)

In [ ]:
fig = plt.figure()
traj_by_label = {
    "Ground Truth": traj_gt,
    "Odometry": traj_odometry,
    "Estimated": traj_estimated
}
ep.trajectories(fig, traj_by_label, ep.PlotMode.xy)
plt.show()

In [ ]:
# Crear figura con 3 subgráficos
fig, axarr = plt.subplots(3, figsize=(10, 6), sharex=True)

# Graficar cada componente en función del tiempo
ep.traj_xyz(axarr, traj_gt, style='-', color='blue', label='Real Trajectory', start_timestamp=traj_gt.timestamps[0])
ep.traj_xyz(axarr, traj_odometry, style='-', color='red', label='Odometry Trajectory', start_timestamp=traj_odometry.timestamps[0])
ep.traj_xyz(axarr, traj_estimated, style='-', color='green', label='Corrected Trajectory', start_timestamp=traj_estimated.timestamps[0])

# Ajustes finales
plt.xlabel("Tiempo (s)")
plt.suptitle("Componentes X, Y, Z de la Trayectoria en función del Tiempo")
plt.show()

In [ ]:
# Crear figura con 3 subgráficos
fig, axarr = plt.subplots(3, figsize=(10, 6), sharex=True)


# Graficar cada componente en función del tiempo
ep.traj_rpy(axarr, traj_gt, style='-', color='blue', label='Real Trajectory', start_timestamp=traj_gt.timestamps[0])
ep.traj_rpy(axarr, traj_odometry, style='-', color='red', label='Odometry Trajectory', start_timestamp=traj_odometry.timestamps[0])
ep.traj_rpy(axarr, traj_estimated, style='-', color='green', label='Corrected Trajectory', start_timestamp=traj_estimated.timestamps[0])

# Ajustes finales
plt.xlabel("Tiempo (s)")
plt.suptitle("Componentes Roll, Pitch, Yaw de la Trayectoria en función del Tiempo")
plt.show()

APE

In [ ]:
# Settings
pose_relation = metrics.PoseRelation.translation_part
# Data preparation
data = (traj_gt, traj_est_aligned)
ape_metric = metrics.APE(pose_relation)
ape_metric.process_data(data)
ape_stat = ape_metric.get_statistic(metrics.StatisticsType.rmse)
print(ape_stat)

In [ ]:
ape_stats = ape_metric.get_all_statistics()
pprint.pprint(ape_stats)

In [ ]:
seconds_from_start = [t - traj_est_aligned.timestamps[0] for t in traj_est_aligned.timestamps]
fig = plt.figure()
ep.error_array(fig.gca(), ape_metric.error, x_array=seconds_from_start,
                 statistics={s:v for s,v in ape_stats.items() if s != "sse"},
                 name="APE", title="APE w.r.t. " + ape_metric.pose_relation.value, xlabel="$t$ (s)")
plt.show()

In [ ]:
plot_mode = ep.PlotMode.xy
fig = plt.figure()
ax = ep.prepare_axis(fig, plot_mode)
ep.traj(ax, plot_mode, traj_gt, '--', "gray", "reference")
ep.traj_colormap(ax, traj_estimated, ape_metric.error, 
                   plot_mode, min_map=ape_stats["min"], max_map=ape_stats["max"])
ax.legend()
plt.show()

In [ ]:
tum_ate_equivalent = metrics.APE(metrics.PoseRelation.translation_part)
tum_ate_equivalent.process_data((traj_gt, traj_est_aligned))
print(tum_ate_equivalent.get_statistic(metrics.StatisticsType.rmse))

## Resultados de APE (Absolute Pose Error)

El APE (Absolute Pose Error) es una métrica utilizada para evaluar la precisión de una trayectoria estimada en comparación con una trayectoria de referencia (ground truth). En este caso, se ha calculado el APE entre la trayectoria estimada alineada y la trayectoria de referencia.

### Estadísticas del APE
Las estadísticas del APE incluyen varias medidas que describen la distribución de los errores de pose absolutos. Algunas de las estadísticas más relevantes son:

- **RMSE (Root Mean Square Error)**: Es una medida de la magnitud promedio del error. Se calcula como la raíz cuadrada de la media de los errores al cuadrado. Un RMSE más bajo indica una mayor precisión.
- **Mean**: Es el valor promedio de los errores. Proporciona una idea de la desviación promedio de la trayectoria estimada respecto a la trayectoria de referencia.
- **Median**: Es el valor mediano de los errores. Es menos sensible a valores atípicos en comparación con la media.
- **Min**: Es el valor mínimo de los errores. Indica el error más pequeño observado.
- **Max**: Es el valor máximo de los errores. Indica el error más grande observado.
- **Std (Standard Deviation)**: Es una medida de la dispersión de los errores. Un valor más bajo indica que los errores están más concentrados alrededor de la media.

### Visualización del APE
Se han generado varias visualizaciones para analizar el APE:

1. **Gráfico de errores en función del tiempo**: Muestra cómo varía el error absoluto de pose a lo largo del tiempo. Esto ayuda a identificar si hay momentos específicos en los que el error es mayor.
2. **Mapa de colores de la trayectoria**: Muestra la trayectoria estimada coloreada según el valor del error en cada punto. Esto permite identificar visualmente las áreas de la trayectoria con mayores errores.

### Interpretación de los Resultados
Los resultados del APE proporcionan una visión detallada de la precisión de la trayectoria estimada. Un RMSE bajo y una desviación estándar baja son indicativos de una trayectoria estimada que sigue de cerca la trayectoria de referencia. Las visualizaciones ayudan a identificar patrones y áreas problemáticas en la estimación de la trayectoria.

En resumen, el análisis del APE es fundamental para evaluar y mejorar la precisión de algoritmos de estimación de trayectoria en aplicaciones de navegación y robótica.

RPE

In [ ]:
pose_relation = metrics.PoseRelation.rotation_angle_deg

# normal mode
delta = 1
delta_unit = Unit.frames

# all pairs mode
all_pairs = False  # activate

In [ ]:

data = (traj_gt, traj_estimated)

In [ ]:
rpe_metric = metrics.RPE(pose_relation=pose_relation, delta=delta, delta_unit=delta_unit, all_pairs=all_pairs)
rpe_metric.process_data(data)

In [ ]:
rpe_stat = rpe_metric.get_statistic(metrics.StatisticsType.rmse)
print(rpe_stat)

In [ ]:
rpe_stats = rpe_metric.get_all_statistics()
pprint.pprint(rpe_stats)

In [ ]:
traj_ref_plot = copy.deepcopy(traj_gt)
traj_est_plot = copy.deepcopy(traj_estimated)
traj_ref_plot.reduce_to_ids(rpe_metric.delta_ids)
traj_est_plot.reduce_to_ids(rpe_metric.delta_ids)
seconds_from_start = [t - traj_estimated.timestamps[0] for t in traj_estimated.timestamps[1:]]

In [ ]:
fig = plt.figure()
ep.error_array(fig.gca(), rpe_metric.error, x_array=seconds_from_start,
                 statistics={s:v for s,v in rpe_stats.items() if s != "sse"},
                 name="RPE", title="RPE w.r.t. " + rpe_metric.pose_relation.value, xlabel="$t$ (s)")
plt.show()

In [ ]:
plot_mode = ep.PlotMode.xy
fig = plt.figure()
ax = ep.prepare_axis(fig, plot_mode)
ep.traj(ax, plot_mode, traj_ref_plot, '--', "gray", "reference")
ep.traj_colormap(ax, traj_est_plot, rpe_metric.error, plot_mode, min_map=rpe_stats["min"], max_map=rpe_stats["max"])
ax.legend()
plt.show()

In [ ]:
tum_rpe_equivalent = metrics.RPE(metrics.PoseRelation.translation_part, delta, Unit.frames, all_pairs=True)
tum_rpe_equivalent.process_data((traj_gt, traj_estimated))
print(tum_rpe_equivalent.get_statistic(metrics.StatisticsType.mean))